# **Reciprocal space and Brillouin zone**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/brillouin_zone.ipynb
<p style="text-align: justify;font-size:15px">
    In this notebook we explore two central concepts in electronic structure and band theory: those of reciprocal space and the Brillouin zone.
</p>
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**
* Appreciate the nature of the relationship between the direct and reciprocal space descriptions of a crystalline system.
* Use this understanding to explain the differences in the Brillouin zones observed for different crystal structures. 


<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
from widget_bzvisualizer import BZVisualizer
import numpy as np
import seekpath
import nglview as nv
from ase.build import bulk, molecule
from ipywidgets import HBox, VBox, Button, Output, Text, Tab, Layout, Label, HTML
import ipywidgets as widgets
import nglview as nv
from ase import Atom, Atoms
from ase.symbols import symbols2numbers
from ase.data import atomic_numbers

In [ ]:
bt_compute = Button(description="Upate")

mat = bulk('Cu', 'fcc', a=3.6)
mat = mat.repeat([3, 3, 3])

m = nv.NGLWidget(width='400px', height='400px')
m.background='black'
c1 = m.add_component(nv.ASEStructure(mat))
m.clear()
m.add_ball_and_stick()
m.add_unitcell()

mpos = [];
msym = [];

for i in mat:
    mpos.append(i.position.tolist())
    msym.append('Cu')


In [ ]:
w = BZVisualizer(mat.cell.tolist(), mpos, symbols2numbers(mat.get_chemical_symbols()), 
                 True, height='400px')

tab_contents = ['a', 'b', 'c']
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children

tab.set_title(0, 'a')
tab.set_title(1, 'b')
tab.set_title(2, 'c')

tab.children[0].value = str(mat.cell[0].tolist())
tab.children[1].value = str(mat.cell[1].tolist())
tab.children[2].value = str(mat.cell[2].tolist())

bt_add = Button(description = "Add")
bt_clear = Button(description = "Clear all")

atom_pos = Text(description="Atom position")
atom_num = Text(description="Atom symbol", layout=Layout(width='120px'))

output = Output(layout=Layout(width="400px"))

ed_index = Text(description="Index", layout=Layout(width='120px'))
ed_sym = Text(description="Atom symbol", layout=Layout(width='120px'))
ed_pos = Text(description="Atom position")
ed_get = Button(description="Get")
ed_set = Button(description="Set")

def update_elements(c):
    with output:
        output.clear_output()
        for i,j in enumerate(mpos):
            print(i, msym[i], j)

update_elements('init')

def add_atom(c):
    global mpos
    
    a = list(eval(atom_pos.value))
    b = atom_num.value
    
    assert type(a) == list
    assert type(b) == str
    
    mpos.append(a)
    msym.append(b)
    atom_pos.value = ''
    atom_num.value = ''
    mat.append(Atom(b, a))
    update_elements('add')
    
bt_add.on_click(add_atom)

def clear_atom(c):
    global mpos, msym
    mpos= [];
    msym = [];
    
    for i in range(len(mat)):
        mat.pop()
        
    update_elements('clear')
    
bt_clear.on_click(clear_atom)

def get_atom(c):
    ed_sym.value = mat.get_chemical_symbols()[int(ed_index.value)]
    ed_pos.value = str(mat[int(ed_index.value)].position.tolist())
    
ed_get.on_click(get_atom)

def set_atom(c):
    mat[int(ed_index.value)].symbol = ed_sym.value
    mat[int(ed_index.value)].position = np.array(eval(ed_pos.value))
    
    msym[int(ed_index.value)] = ed_sym.value
    mpos[int(ed_index.value)] = np.array(eval(ed_pos.value))
    
    ed_sym.value = ''
    ed_pos.value = ''
    
    update_elements('set')
    
ed_set.on_click(set_atom)
    
tab2 = Tab()
tab2.children = [VBox([HBox([atom_num, atom_pos]), HBox([bt_add, bt_clear])]), 
                 VBox([HBox([ed_sym, ed_pos]), HBox([ed_index, ed_get, ed_set])]), output]

tab2.set_title(0, 'Add atoms')
tab2.set_title(1, 'Edit atoms')
tab2.set_title(2, 'List of atoms')

def compute_BZ(c):
    global c1
    v1 = list(eval(tab.children[0].value))
    v2 = list(eval(tab.children[1].value))
    v3 = list(eval(tab.children[2].value))
    
    assert type(v1) == list
    assert type(v2) == list
    assert type(v3) == list
    
    mat.set_cell([v1, v2, v3])
    m.remove_component(c1)
    c1 = m.add_component(nv.ASEStructure(mat))
    m.clear()
    m.add_ball_and_stick()
    m.add_unitcell()
    w.numbers = symbols2numbers(mat.get_chemical_symbols())
    w.positions = mpos
    w.cell = mat.cell.tolist()
    w.update_structure += 1


bt_compute.on_click(compute_BZ);

label1 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='black'><font size=5>Structure</b></div>")
label2 = HTML(value = f"<div style='width: 400px; text-align:center;'><b><font color='black'><font size=5>Brillouin zone</b></div>")

In [ ]:
display(HBox([label1, label2]), HBox([m, w]), HBox([tab, tab2]), bt_compute)